In [1]:
import os
import json

# directory where JSON files are located
directory = os.getcwd()

# get all files in the current working directory
files = os.listdir(directory)

# filter out only JSON files
json_files = [file for file in files if file.endswith(".json")]

# initialize a dictionary to store the data from all the JSON files
data = {}

# helper function to merge two dictionaries with nested lists


def merge_dicts(dict1, dict2):
    for key, value in dict2.items():
        if key in dict1 and isinstance(value, (list, dict)):
            # if the key exists in both dictionaries and the value is a list or dictionary, recursively merge the nested data
            if isinstance(value, list):
                # if the value is a list, merge the lists
                dict1[key] = dict1[key] + value
            else:
                # if the value is a dictionary, recursively merge the dictionaries
                dict1[key] = merge_dicts(dict1[key], value)
        else:
            # otherwise, just add the key-value pair from dict2 to dict1
            dict1[key] = value
    return dict1


# load and parse each JSON file, and merge the data into the dictionary
for file in json_files:
    with open(os.path.join(directory, file)) as json_file:
        file_data = json.load(json_file)
        data = merge_dicts(data, file_data)

if not os.path.exists("output"):
    os.makedirs("output")

# save the merged data to a new JSON file
with open("output/merged_data.json", "w") as outfile:
    json.dump(data, outfile)

data = json.load(open(os.path.join(directory, "output/merged_data.json")))
    



In [2]:
def score_path(path,trails,tree):
    score = 0
    for trail in path:
        score +=trails[trail]["distance"]
    return score
    

In [3]:
def decode(encoded):
    path = []
    n = [0, 0]
    j = 0
    for i in range(len(encoded)):
        k = ord(encoded[i]) - 0x30
        n[j] = (n[j] << 5) | (k & 0x1f)
        if k & 0x20:
            n[j] = (~(n[j] >> 1) if n[j] & 0x01 else n[j] >> 1) * 1e-7
            if j == 1:
                latLon = path[-1] if len(path) > 0 else (0, 0)
                path.append((latLon[0] + n[0], latLon[1] + n[1]))
            j = (j + 1) % 2
            n[j] = 0
    return (path[0], path[-1])

In [4]:
for trail in data["trails"].values():
    # print(trail)
    startPos, endPos = decode(trail.pop("encoded"))
    trail["start_latlng"] = (round(startPos[0], 6), round(startPos[1], 6))
    trail["end_latlng"] = (round(endPos[0], 6), round(endPos[1], 6))


In [5]:
import numpy as np
print(len(data["nodes"]))
data["nodes"] = {key: value for (key, value) in data["nodes"].items() if value["region"] == "Tatry"}
print(len(data["nodes"]))

start_points = ["Siwa Polana", "Polana Biały Potok", "Kiry", "Staników Żleb", "Nędzówka, las", "Gronik", "Za Bramką", "Dolina Strążyska, parking",
                "Dolina ku Dziurze", "Dolina Białego", "Kuźnicka Polana", "Kuźnice", "Jaszczurówka", "Lysá Poľana, parking", "Palenica Białczańska", "Tatranská Javorina", "Monkova dolina", "Tatranská Kotlina", "Nad Matliarmi", "Hrebienok", "Velický most", "Nad Zrubami", "Trigan", "Pod Furkotskou doliou", "Pod Grúnikom", "Podbanské, rozc.", "Úzka dolina, parking", "Pod Holým vrchom", "Žiarska dolina, ústie", "Jalovecká dolina, ústie", "Biela skala, horáreň", "Brestová, skanzen", "Rázcestie pod Suchou dolinou"]

start_points_ids = []

for i,node in enumerate(data["nodes"].values()):
    node["id"]=i
    if node["name"] in start_points:
        start_points_ids.append(i)

print(start_points_ids)


matrix = np.full((len(data["nodes"]), len(data["nodes"])),np.inf)
for node in data["nodes"].values():
    for trail in data["trails"].values():
        if trail["start_latlng"] == (node["latitude"], node["longitude"]):
            for node2 in data["nodes"].values():
                if trail["end_latlng"] == (node2["latitude"], node2["longitude"]):
                    matrix[node["id"], node2["id"]] = trail["start_end_time"]
                    matrix[node2["id"], node["id"]] = trail["end_start_time"]
        if trail["end_latlng"] == (node["latitude"], node["longitude"]):
            for node2 in data["nodes"].values():
                if trail["start_latlng"] == (node2["latitude"], node2["longitude"]):
                    matrix[node["id"], node2["id"]] = trail["start_end_time"]
                    matrix[node2["id"], node["id"]] = trail["end_start_time"]

# print(matrix)
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
# with open("matrix.txt", 'w') as f:
    # f.write(np.array2string(matrix, separator=', '))

        

764
336
[4, 5, 11, 23, 47, 50, 72, 76, 81, 92, 102, 115, 124, 144, 150, 162, 173, 175, 177, 188, 200, 207, 245, 247, 286, 312, 317, 319, 329, 332, 333]


In [6]:
start_points = ["Siwa Polana", "Polana Biały Potok", "Kiry", "Staników Żleb", "Nędzówka, las", "Gronik", "Za Bramką", "Dolina Strążyska, parking",
                "Dolina ku Dziurze", "Dolina Białego", "Kuźnicka Polana", "Kuźnice", "Jaszczurówka", "Lysá Poľana, parking", "Palenica Białczańska", "Tatranská Javorina", "Monkova dolina", "Tatranská Kotlina", ""]

for point in start_points:
    point=point.encode("UTF-16")

In [7]:
print(np.count_nonzero(matrix))
print(len(data["trails"].values()))


112896
971


In [18]:
nodes = data["nodes"].values()
good_list = []
for node in nodes:
    good_list.append(node)
# print(good_list)
route = [144, 325, 225, 134, 274, 163, 159, 212, 158, 211, 237, 227, 237, 227, 237, 211, 199, 299, 143, 192, 143, 290, 272, 295, 178, 201, 176, 201, 126, 139, 126, 201, 178, 201, 178, 201, 176, 198, 178, 295, 178, 198, 176, 197, 176, 198, 176, 201, 178, 198, 178, 295, 272, 290, 168, 290, 168, 290, 272, 173, 333, 320, 170, 221, 239, 206, 298, 303, 112, 128, 142, 105, 135, 219, 149, 203, 256, 203, 280, 189, 280, 203, 149, 275, 149, 203, 256, 203, 256, 268, 256, 189, 256, 268, 256, 189, 114, 209, 308, 189, 114, 209, 297, 122, 160, 122, 297, 209, 114, 189, 280, 307, 302, 240, 300, 238, 132, 214, 99, 116, 196, 314, 196, 116, 99, 116, 196, 116, 99, 214, 132, 238, 190, 238, 300, 238, 190, 238, 214, 132, 238, 132, 214, 238, 300, 253, 243, 302, 307, 140, 242, 182, 259, 191, 313, 311, 309, 311, 292, 145, 292, 241, 292, 311, 313, 191, 259, 309, 311, 292, 241, 292, 241, 145, 131, 174, 281, 174, 109, 171, 109, 174, 131, 145, 131, 121, 171, 121, 171, 121, 131, 174, 131, 121, 131, 174, 281, 174, 281, 216, 243, 302, 240, 302, 243, 302, 240, 302, 243, 253, 300, 238, 300, 253, 243, 216, 243, 216, 281, 244, 161, 244, 284, 179, 246, 165, 246, 179, 246, 179, 229, 148, 229, 179, 246, 277, 115, 319, 322, 326, 142, 128, 112, 282, 112, 303, 298, 251, 215, 127, 248, 210, 164, 210, 164, 210, 248, 289, 285, 135, 105, 135, 219, 135, 105, 142, 128, 266, 324, 329, 11, 59, 17, 73, 26, 31, 43, 31, 44, 71, 44, 31, 43, 31, 43, 31, 44, 71, 41, 54, 57, 49, 27, 21, 20, 264, 233, 166, 152, 263, 218, 276, 218, 263, 152, 166, 233, 264, 233, 166, 152, 270, 183, 32, 90, 32, 38, 54, 38, 54, 57, 46, 78, 27, 78, 27, 21, 20, 264, 183, 270, 152, 263, 155, 276, 218, 263, 218, 226, 261, 233, 261, 226, 218, 276, 269, 278, 269, 304, 39, 157, 39, 304, 39, 157, 255, 125, 213, 288, 213, 184, 172, 278, 269, 276, 218, 226, 273, 261, 233, 166, 152, 263, 155, 270, 183, 270, 152, 166, 152, 263, 218, 276, 155, 296, 185, 296, 224, 250, 274, 279, 167, 279, 274, 279, 167, 262, 167, 279, 274, 250, 274, 163, 167, 279, 274, 134, 225, 134, 274, 250, 287, 274, 250, 287, 235, 153, 172, 278, 269, 276, 218, 276, 269, 278, 147, 180, 147, 180, 235, 287, 274, 250, 287, 235, 180, 235, 287, 250, 287, 235, 287, 250, 224, 180, 235, 287, 250, 274, 279, 167, 262, 228, 200, 333, 320, 170, 320, 170, 221, 104, 298, 251, 215, 129, 215, 127, 164, 210, 113, 217, 208, 181, 101, 217, 113, 217, 101, 181, 120, 129, 215, 127, 248, 210, 297, 209, 297, 210, 113, 217, 101, 181, 208, 217, 101, 164, 127, 164, 127, 215, 127, 248, 127, 215, 251, 138, 108, 187, 111, 108, 187, 137, 187, 108, 138, 251, 298, 303, 112, 282, 123, 282, 112, 282, 123, 105, 142, 128, 266, 319, 72, 61, 75, 61, 75, 61, 75, 88, 30, 42, 30, 88, 70, 95, 64, 94, 40, 52, 48, 58, 91, 45, 37, 97, 18, 97, 12, 69, 12, 97, 18, 15, 81, 188, 305, 306, 146, 306, 294, 146, 294, 306, 294, 146, 301, 133, 232, 125, 232, 133, 301, 146, 306, 294, 146, 301, 133, 232, 125, 255, 260, 157, 39, 157, 39, 20, 264, 183, 32, 38, 54, 41, 71, 44, 31, 43, 31, 43, 31, 43, 31, 26, 73, 26, 31, 44, 71, 41, 54, 57, 46, 63, 46, 78, 16, 55, 60, 77, 80, 77, 60, 23, 188, 247, 242, 140, 307, 280, 203, 149, 203, 280, 189, 308, 205, 289, 248, 127, 215, 251, 215, 129, 120, 220, 103, 110, 118, 103, 110, 118, 103, 118, 110, 103, 220, 120, 220, 117, 220, 120, 181, 208, 141, 117, 220, 120, 220, 120, 181, 208, 141, 117, 220, 103, 110, 118, 103, 118, 110, 103, 118, 110, 103, 118, 110, 118, 103, 110, 118, 110, 118, 103, 110, 103, 220, 103, 220, 120, 181, 208, 181, 120, 129, 111, 129, 120, 181, 101, 217, 113, 210, 164, 127, 215, 251, 298, 303, 298, 251, 138, 251, 298, 206, 239, 137, 187, 137, 239, 137, 239, 206, 298, 104, 298, 303, 112, 128, 142, 105, 135, 219, 149, 219, 135, 285, 123, 105, 142, 326, 322, 334, 322, 319, 76, 24, 96, 82, 85, 76, 92, 13, 84, 28, 86, 66, 35, 66, 86, 28, 84, 13, 23, 5, 10, 35, 83, 22, 83, 35, 9, 0, 9, 0, 7, 2, 53, 51, 1, 8, 51, 1, 51, 53, 59, 17, 73, 26, 31, 44, 71, 44, 71, 41, 71, 44, 71, 41, 54, 38, 56, 38, 56, 38, 54, 57, 46, 57, 49, 21, 27, 49, 27, 21, 49, 57, 46, 78, 16, 23, 188, 305, 271, 202, 156, 200, 144, 100, 106, 100, 257, 231, 130, 231, 136, 310, 136, 134, 274, 287, 235, 180, 147, 186, 147, 186, 147, 180, 224, 296, 185, 296, 155, 263, 155, 296, 155, 276, 269, 278, 269, 304, 236, 273, 261, 233, 264, 183, 270, 155, 296, 224, 180, 224, 296, 155, 263, 152, 166, 233, 264, 20, 21, 27, 78, 16, 78, 46, 57, 49, 27, 78, 46, 57, 49, 21, 49, 57, 46, 57, 49, 57, 46, 63, 46, 78, 27, 49, 21, 27, 78, 27, 49, 21, 20, 39, 157, 39, 20, 264, 183, 32, 90, 32, 38, 54, 57, 46, 63, 46, 57, 49, 27, 49, 21, 20, 264, 20, 39, 157, 260, 34, 232, 34, 232, 133, 301, 146, 306, 305, 188, 50, 62, 37, 45, 37, 45, 91, 58, 48, 52, 48, 58, 91, 45, 37, 45, 91, 58, 87, 58, 48, 52, 40, 94, 40, 94, 40, 52, 48, 58, 87, 58, 48, 93, 89, 91, 58, 87, 58, 48, 52, 40, 94, 64, 98, 96, 24, 76, 102, 160, 193, 107, 102, 144, 100, 257, 33, 67, 33, 17, 59, 8, 11, 245, 299, 199, 299, 143, 192, 143, 299, 199, 211, 158, 293, 223, 165, 246, 179, 284, 179, 284, 244, 284, 244, 161, 244, 284, 179, 284, 179, 229, 148, 249, 212, 159, 197, 158, 293, 151, 222, 223, 293, 158, 211, 199, 151, 293, 223, 222, 207, 317, 328, 321, 154, 327, 154, 321, 328, 317, 188, 305, 271, 202, 291, 202, 271, 169, 162, 332, 330, 265, 116, 99, 214, 99, 214, 238, 132, 238, 300, 240, 300, 238, 190, 275, 149, 219, 149, 219, 135, 219, 323, 326, 142, 128, 104, 221, 239, 206, 239, 221, 104, 128, 142, 326, 323, 119, 275, 190, 275, 149, 219, 135, 219, 323, 119, 132, 238, 190, 275, 149, 275, 119, 132, 214, 99, 214, 99, 116, 99, 214, 99, 116, 196, 314, 196, 116, 196, 116, 265, 330, 265, 116, 99, 116, 196, 314, 332, 177, 165, 283, 249, 212, 249, 283, 165, 177, 5, 10, 35, 66, 86, 14, 42, 74, 24, 98, 64, 94, 64, 94, 40, 52, 48, 58, 87, 58, 91, 58, 91, 58, 87, 58, 91, 45, 29, 45, 91, 45, 37, 62, 89, 93, 89, 62, 50, 247, 242, 140, 307, 140, 307, 280, 203, 280, 307, 302, 307, 140, 252, 140, 252, 140, 307, 302, 243, 216, 281, 174, 109, 195, 109, 174, 131, 145, 241, 292, 311, 309, 267, 309, 311, 292, 241, 145, 292, 145, 292, 241, 292, 311, 309, 259, 182, 277, 115, 175, 177, 286, 245, 247, 150, 333, 335, 329, 23, 16, 55, 19, 34, 232, 125, 213, 184, 204, 288, 213, 288, 291, 156, 200, 312, 207, 124, 177, 162, 234, 162, 317, 318, 254, 318, 6, 3, 6, 318, 254, 65, 254, 318, 317, 333, 331, 333, 329, 316, 319, 332, 330, 315, 334, 322, 319, 245, 312, 286, 194, 286, 173, 286, 81, 15, 18, 61, 72, 200, 162, 329, 324, 335, 333, 150, 252, 150, 247, 115, 4, 10, 35, 66, 25, 66, 25, 66, 25, 66, 86, 14, 86, 66, 25, 66, 35, 10, 5, 47, 36, 47, 312, 230, 312, 317, 328, 321, 154, 258, 154, 258, 154, 258, 154, 321, 328, 317, 173, 272, 295, 178, 201, 176, 201, 178, 198, 176, 197, 159, 197, 176, 201, 126, 228, 126, 201, 176, 198, 178, 201, 176, 198, 176, 198, 178, 198, 176, 201, 178, 295, 272, 290, 143, 299, 245, 23, 60, 55, 60, 77, 60, 55, 19, 80, 77, 60, 23, 200, 156, 291, 288, 204, 153, 172, 153, 204, 288, 291, 202, 156, 291, 288, 213, 125, 255, 260, 34, 260, 255, 125, 232, 133, 301, 146, 306, 305, 188]
output = []
for element in route:
    output.append((good_list[element]['latitude'],good_list[element]['longitude']))
formatted_output = ""
for element in output:
    formatted_output+=f"{element[0]},{element[1]};"
print(formatted_output)

49.15509,49.15509;49.151248,49.151248;49.200399,49.200399;49.222358,49.222358;49.23099,49.23099;49.231336,49.231336;49.234471,49.234471;49.236504,49.236504;49.246364,49.246364;49.248611,49.248611;49.250304,49.250304;49.251005,49.251005;49.250304,49.250304;49.251005,49.251005;49.250304,49.250304;49.248611,49.248611;49.258317,49.258317;49.263028,49.263028;49.261913,49.261913;49.259355,49.259355;49.261913,49.261913;49.262804,49.262804;49.261394,49.261394;49.262629,49.262629;49.260931,49.260931;49.263934,49.263934;49.256331,49.256331;49.263934,49.263934;49.264497,49.264497;49.265389,49.265389;49.264497,49.264497;49.263934,49.263934;49.260931,49.260931;49.263934,49.263934;49.260931,49.260931;49.263934,49.263934;49.256331,49.256331;49.259452,49.259452;49.260931,49.260931;49.262629,49.262629;49.260931,49.260931;49.259452,49.259452;49.256331,49.256331;49.249647,49.249647;49.256331,49.256331;49.259452,49.259452;49.256331,49.256331;49.263934,49.263934;49.260931,49.260931;49.259452,49.259452;49.2